In [1]:
# PATH = '/kaggle/input/utilities-cifar10'

# import sys

# sys.path.insert(1, PATH)

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import random 
import torch
import torchvision
from torchvision import transforms, datasets
from torch.autograd import Variable
from torch.utils.data import DataLoader,random_split
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
import time

from ae_cifar10 import AutoEncoder
from utils import fgsm_attack, pgd_linf, to_numpy_array_cifar10, EarlyStopping
from vgg import VGG

c:\Users\malay\miniconda3\envs\cs776\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
attack_type = 'fgsm'
activation_type = 'gelu'
lr = 0.001
batch_size = 16
num_epochs = 10
dataset = 'cifar10'
# z_dim = 10
include_noise = True

# FGSM parameters
eps_fgsm = 5e-3
#PGD parameters
eps_pgd, alpha, num_iter = 5e-3, 1e-2, 5

In [4]:
# CIFAR10 Dataset
transform = transforms.Compose(
    [transforms.Resize((224, 224)),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

transform_ae = transforms.Compose(
    [transforms.Resize((32, 32))])

transform_classifier = transforms.Compose(
    [transforms.Resize((224, 224))])

dataset = datasets.CIFAR10(root= './data', train = True, download =True, transform = transform)
# len(dataset)
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [40000, 10000])

Files already downloaded and verified


In [5]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f'Selected device: {device}')

Selected device: cuda


In [6]:
classifier = VGG()
classifier.to(device)
classifier.load_state_dict(torch.load('./models/vgg16.pth.tar')['vgg16_state_dict'])

<All keys matched successfully>

In [7]:
# if activation_type == 'gelu':
#     convAE = Autoencoder_GELU(device, include_noise)
# else:
#     convAE = Autoencoder_RELU(device, include_noise)

convAE = AutoEncoder(device, include_noise)
convAE.to(device)
# Loss function
criterion = nn.MSELoss()
# Optimizer
optimizer = optim.Adam(convAE.parameters(), lr=lr)
# Early stopping criteria
early_stopping = EarlyStopping()
# Check the model
convAE

AutoEncoder(
  (encoder): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): GELU(approximate='none')
    (2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): GELU(approximate='none')
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (6): GELU(approximate='none')
    (7): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (8): GELU(approximate='none')
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (decoder): Sequential(
    (0): ConvTranspose2d(64, 32, kernel_size=(2, 2), stride=(2, 2))
    (1): GELU(approximate='none')
    (2): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (3): GELU(approximate='none')
    (4): ConvTranspose2d(32, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (5): Conv2d(16, 16, kernel_size

In [8]:
from torchsummary import summary

summary(convAE, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 32, 32]             448
              GELU-2           [-1, 16, 32, 32]               0
            Conv2d-3           [-1, 32, 32, 32]           4,640
              GELU-4           [-1, 32, 32, 32]               0
         MaxPool2d-5           [-1, 32, 16, 16]               0
            Conv2d-6           [-1, 32, 16, 16]          25,632
              GELU-7           [-1, 32, 16, 16]               0
            Conv2d-8           [-1, 64, 16, 16]          51,264
              GELU-9           [-1, 64, 16, 16]               0
        MaxPool2d-10             [-1, 64, 8, 8]               0
  ConvTranspose2d-11           [-1, 32, 16, 16]           8,224
             GELU-12           [-1, 32, 16, 16]               0
           Conv2d-13           [-1, 32, 16, 16]          25,632
             GELU-14           [-1, 32,

In [7]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

In [21]:
def load_checkpoint(checkpoint):
    convAE.load_state_dict(checkpoint['convAE_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    return checkpoint['train_losses'], checkpoint['val_losses']

def save_checkpoint(checkpoint, model_path):
    torch.save(checkpoint, model_path)

In [28]:
load_model = True
if include_noise:
    file_path = f'./models/ae_cifar10_{attack_type}_{activation_type}_proposed.pth.tar'
else:
    file_path = f'./models/ae_cifar10_{attack_type}_{activation_type}_vanilla.pth.tar'

train_losses, val_losses = [], []

if load_model:
    train_losses, val_losses = load_checkpoint(torch.load(file_path))

len(train_losses), len(val_losses)

(11, 11)

In [16]:
start_time = time.time()

for epoch in range(num_epochs):
    train_loss, val_loss = 0.0, 0.0

    for _, (imgs, labels) in enumerate(train_loader):
        # print(imgs.shape)
        batch_size = imgs.shape[0]
        imgs, labels = Variable(imgs.to(device), requires_grad=True), Variable(labels.to(device))
        
        if attack_type == 'fgsm':
            adv_imgs, _ = fgsm_attack(classifier, imgs, labels, eps_fgsm, dataset)
        else:
            adv_imgs, _ = pgd_linf(classifier, imgs, labels, eps_pgd, alpha, num_iter)
            
        adv_imgs = adv_imgs.to(device)
        imgs, adv_imgs_ = transform_ae(imgs), transform_ae(adv_imgs).to(device)

        optimizer.zero_grad()
        rec_imgs = convAE.forward(adv_imgs_)
        loss = criterion(imgs, rec_imgs)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*batch_size

    for _, (imgs, labels) in enumerate(val_loader):
        # print(imgs.shape)
        batch_size = imgs.shape[0]
        imgs, labels = Variable(imgs.to(device), requires_grad=True), Variable(labels.to(device))

        if attack_type == 'fgsm':
            adv_imgs, _ = fgsm_attack(classifier, imgs, labels, eps_fgsm, dataset)
        else:
            adv_imgs, _ = pgd_linf(classifier, imgs, labels, eps_pgd, alpha, num_iter)

        adv_imgs = adv_imgs.to(device)
        imgs, adv_imgs_ = transform_ae(imgs), transform_ae(adv_imgs).to(device)
        # adv_imgs = adv_imgs.to(device)

        optimizer.zero_grad()
        rec_imgs = convAE.forward(adv_imgs_)
        loss = criterion(imgs, rec_imgs)
        loss.backward()
        optimizer.step()
        val_loss += loss.item()*batch_size
    
    train_loss, val_loss = train_loss/len(train_dataset), val_loss/len(val_dataset)

    train_losses.append(train_loss)
    val_losses.append(val_loss)

    print(f'Epoch: {epoch+1}/{num_epochs}, train_loss: {train_loss:.4f}, val_loss: {val_loss:.4f}, \
          time_taken: {(time.time()-start_time)/60:.2f} mins')
    
    checkpoint = {
        'convAE_state_dict': convAE.state_dict(), 'optimizer_state_dict': optimizer.state_dict(),
        'train_losses': train_losses, 'val_losses': val_losses
    }

    save_checkpoint(checkpoint, file_path)

    early_stopping(train_loss, val_loss)
    if early_stopping.early_stop:
        print("Early Stopping critieria satisfied")
        break

Epoch: 1/10, train_loss: 0.0821, val_loss: 0.0810,           time_taken: 35.72 mins


KeyboardInterrupt: 